# AutoAWQ를 이용한 quantization

- vllm에서 지원하는 quantization 형식은 AWQ
- 대표적인 라이브러리 AutoAWQ를 사용해 양자화 진행

## About AutoAWQ
- 기본적으로 CUDA 12.1 지원 -> 그 이하일 경우 아래 명령어 입력
```bash
!pip install -q -U https://github.com/casper-hansen/AutoAWQ/releases/download/v0.1.6/autoawq-0.1.6+cu118-cp310-cp310-linux_x86_64.whl
```

In [1]:
!nvcc -V # CUDA 버전 확인

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2023 NVIDIA Corporation
Built on Tue_Aug_15_22:02:13_PDT_2023
Cuda compilation tools, release 12.2, V12.2.140
Build cuda_12.2.r12.2/compiler.33191640_0


In [2]:
# 패키지 설치
!pip install -q transformers autoawq pillow torch torchvision torchaudio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.9/94.9 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 779.1/779.1 MB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 115.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 95.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 42.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 16.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 37.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.0/196.0 MB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 176.2/176.2 MB 11.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [3]:
!pwd

/content


In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
!cd /content/drive/MyDrive/HelloWorld-AI/HelloWorld-AI/model

In [7]:
import os
os.makedirs("./quantized_model", exist_ok=True)

In [8]:
from awq import AutoAWQForCausalLM
from transformers import AutoTokenizer

model_path = "Bllossom/llama-3.1-Korean-Bllossom-Vision-8B"
quant_path = "Bllossom/llama-3.1-Korean-Bllossom-Vision-8B-awq"
quant_config = {"zero_point": True, "q_group_size": 128, "w_bit": 4, "version":"GEMM"} # default setting

# Load model
model = AutoAWQForCausalLM.from_pretrained(model_path, device_map="auto",
                                            cache_dir='../cache',
                                            trust_remote_code=True)
tokenizer = AutoTokenizer.from_pretrained(model_path, trust_remote_code=True)

config.json:   0%|          | 0.00/1.35k [00:00<?, ?B/s]

Fetching 13 files:   0%|          | 0/13 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/143 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/9.53k [00:00<?, ?B/s]

.gitattributes:   0%|          | 0.00/1.52k [00:00<?, ?B/s]

...

model.safetensors.index.json:   0%|          | 0.00/70.2k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/444 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/51.7k [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [9]:
import logging
logging.basicConfig(level=logging.DEBUG)

# Quantize
model.quantize(tokenizer, quant_config=quant_config)

README.md:   0%|          | 0.00/167 [00:00<?, ?B/s]

Repo card metadata block was not found. Setting CardData to empty.


val.jsonl.zst:   0%|          | 0.00/471M [00:00<?, ?B/s]

Generating validation split:   0%|          | 0/214670 [00:00<?, ? examples/s]

AWQ: 100%|██████████| 32/32 [29:41<00:00, 55.66s/it]


In [11]:
from transformers import AwqConfig, AutoConfig
from huggingface_hub import HfApi
import os

# modify the config file so that it is compatible with transformers integration
quantization_config = AwqConfig(
    bits=quant_config["w_bit"],
    group_size=quant_config["q_group_size"],
    zero_point=quant_config["zero_point"],
    version=quant_config["version"].lower(),
).to_dict()

# the pretrained transformers model is stored in the model attribute + we need to pass a dict
model.model.config.quantization_config = quantization_config
# a second solution would be to use Autoconfig and push to hub (what we do at llm-awq)


# save model weights
model.save_quantized(os.path.join("/content/drive/MyDrive/HelloWorld-AI/HelloWorld-AI/model/quantized_model", quant_path.split("/")[1]))

# save tokenizer
tokenizer.save_pretrained(os.path.join("/content/drive/MyDrive/HelloWorld-AI/HelloWorld-AI/model/quantized_model", quant_path.split("/")[1]))

Note that `shard_checkpoint` is deprecated and will be removed in v4.44. We recommend you using split_torch_state_dict_into_shards from huggingface_hub library


('/content/drive/MyDrive/HelloWorld-AI/HelloWorld-AI/model/quantized_model/llama-3.1-Korean-Bllossom-Vision-8B-awq/tokenizer_config.json',
 '/content/drive/MyDrive/HelloWorld-AI/HelloWorld-AI/model/quantized_model/llama-3.1-Korean-Bllossom-Vision-8B-awq/special_tokens_map.json',
 '/content/drive/MyDrive/HelloWorld-AI/HelloWorld-AI/model/quantized_model/llama-3.1-Korean-Bllossom-Vision-8B-awq/tokenizer.json')

In [ ]:
api = HfApi()
api.upload_folder(
    folder_path="Bllossom/llama-3.1-Korean-Bllossom-Vision-8B-awq",
    repo_id="ywhwang/llama-3.1-Korean-Bllossom-Vision-8B-awq",
    repo_type="model",
)

model-00002-of-00002.safetensors:   0%|          | 0.00/1.38G [00:00<?, ?B/s]

Upload 2 LFS files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/ywhwang/llama-3.1-Korean-Bllossom-Vision-8B-awq/commit/ce36ba3c75a5e04fd996746ecdc54932e6432920', commit_message='Upload folder using huggingface_hub', commit_description='', oid='ce36ba3c75a5e04fd996746ecdc54932e6432920', pr_url=None, pr_revision=None, pr_num=None)

In [ ]:
%%bash
pip install llava flash-attn --no-build-isolation --no-cache-dir
pip install --upgrade transformers

In [ ]:
!pip show transformers

Name: transformers
Version: 4.44.2
Summary: State-of-the-art Machine Learning for JAX, PyTorch and TensorFlow
Home-page: https://github.com/huggingface/transformers
Author: The Hugging Face team (past and future) with the help of all our contributors (https://github.com/huggingface/transformers/graphs/contributors)
Author-email: transformers@huggingface.co
License: Apache 2.0 License
Location: /usr/local/lib/python3.10/dist-packages
Requires: filelock, huggingface-hub, numpy, packaging, pyyaml, regex, requests, safetensors, tokenizers, tqdm
Required-by: autoawq


In [ ]:
from transformers import LlavaNextProcessor, LlavaNextForConditionalGeneration
import torch
import requests

tokenizer = AutoTokenizer.from_pretrained("ywhwang/llama-3.1-Korean-Bllossom-Vision-8B-awq")
processor = LlavaNextProcessor.from_pretrained("ywhwang/llama-3.1-Korean-Bllossom-Vision-8B-awq")

model = LlavaNextForConditionalGeneration.from_pretrained("ywhwang/llama-3.1-Korean-Bllossom-Vision-8B-awq", torch_dtype=torch.float16, low_cpu_mem_usage=True, device_map='auto')
model.eval()


with torch.no_grad():
    PROMPT=\
    """You are a versatile AI assistant named Bllava, capable of both understanding and generating text as well as interpreting and analyzing images. Your role is to kindly and effectively answer the user’s questions, whether they are about text or images, and provide appropriate and helpful responses to all types of queries.

    당신은 텍스트를 이해하고 생성하는 것뿐만 아니라 이미지를 해석하고 분석할 수 있는 다재다능한 AI 어시스턴트 블라바입니다. 사용자의 질문이 텍스트에 관한 것이든 이미지에 관한 것이든 친절하고 효과적으로 답변하며, 모든 유형의 질의에 대해 적절하고 유용한 응답을 제공하는 것이 당신의 역할입니다."""

    instruction = '자연어처리 15주 분량 커리큘럼을 짜줘'

    messages = [
        {'role': 'system', 'content': f"{PROMPT}"},
        {'role': 'user', 'content': f"{instruction}"}
    ]

    chat_messages = processor.tokenizer.apply_chat_template(
        messages,
        tokenize=True,
        add_generation_prompt=True,
        return_tensors='pt',
    )

    bos_token = processor.tokenizer.bos_token_id
    chat_messages = torch.cat([torch.tensor([[bos_token]]),chat_messages],dim=-1).to(model.device)


    output = model.generate(
        input_ids = chat_messages,
        use_cache=False,
        max_new_tokens=2048,
        top_p=0.9,
        temperature=0.6,
        do_sample=True,
    )

    print(processor.tokenizer.decode(output[0]))

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Some weights of the model checkpoint at ywhwang/llama-3.1-Korean-Bllossom-Vision-8B-awq were not used when initializing LlavaNextForConditionalGeneration: ['multi_modal_projector.linear_1.weight', 'multi_modal_projector.linear_2.weight', 'vision_tower.vision_model.encoder.layers.0.mlp.fc1.weight', 'vision_tower.vision_model.encoder.layers.0.mlp.fc2.weight', 'vision_tower.vision_model.encoder.layers.0.self_attn.k_proj.weight', 'vision_tower.vision_model.encoder.layers.0.self_attn.out_proj.weight', 'vision_tower.vision_model.encoder.layers.0.self_attn.q_proj.weight', 'vision_tower.vision_model.encoder.layers.0.self_attn.v_proj.weight', 'vision_tower.vision_model.encoder.layers.1.mlp.fc1.weight', 'vision_tower.vision_model.encoder.layers.1.mlp.fc2.weight', 'vision_tower.vision_model.encoder.layers.1.self_attn.k_proj.weight', 'vision_tower.vision_model.encoder.layers.1.self_attn.out_proj.weight', 'vision_tower.vision_model.encoder.layers.1.self_attn.q_proj.weight', 'vision_tower.vision_mod

<|begin_of_text|><|start_header_id|>system<|end_header_id|>

You are a versatile AI assistant named Bllava, capable of both understanding and generating text as well as interpreting and analyzing images. Your role is to kindly and effectively answer the user’s questions, whether they are about text or images, and provide appropriate and helpful responses to all types of queries.
    
    당신은 텍스트를 이해하고 생성하는 것뿐만 아니라 이미지를 해석하고 분석할 수 있는 다재다능한 AI 어시스턴트 블라바입니다. 사용자의 질문이 텍스트에 관한 것이든 이미지에 관한 것이든 친절하고 효과적으로 답변하며, 모든 유형의 질의에 대해 적절하고 유용한 응답을 제공하는 것이 당신의 역할입니다.<|eot_id|><|start_header_id|>user<|end_header_id|>

자연어처리 15주 분량 커리큘럼을 짜줘<|eot_id|><|start_header_id|>assistant<|end_header_id|>

자연어 처리(NLP) 분야는 매우 광범위하고 발전 속도가 매우 빠르기 때문에, 15주 분량의 커리큘럼을 짜는 것은 도전적인 일입니다. 그러나 주요 주제와 기술을 기반으로 한 커리큘럼을 제공하는 것은 가능합니다. 다음은 자연어 처리에 대한 15주 분량의 학습 경로입니다:

### 주 1: 소개 및 역사
- 자연어 처리의 정의
- 역사
- NLP의 응용 분야

### 주 2: 언어 모델링
- 언어 모델링 개념
- 언어 모델링의 목적
- 예: BERT, GPT

### 주 3: 텍스트 전처리
- 텍스트의 형식과 구조
- 텍스트의 정규화
- 텍스트 처리 도구 (토크